In [1]:
import pandas as pd

In [2]:
## MAIN FUNCTION - input

## INPUT FILE
# codemaps = pd.read_csv('codemaps.csv')
country_names = pd.read_csv('../../../data/dl1_countrycodeorg_country_name.csv')
UNcode_names = pd.read_csv('WPP2019_F01_LOCATIONS_DB.csv')
UNcode_names = UNcode_names[['Country Code', 'LocID']]

df_input_pop = pd.read_csv('WPP2019_PopulationByAgeSex_Medium.csv')
df_input_pop_total = pd.read_csv('WPP2019_TotalPopulationBySex.csv')

start_year = 2010
end_year = 2051
## OUTPUT FILE
save_file_name = "population_un.csv"
save_file_name_total = "population_total.csv"

In [3]:
## Merge data

# ages = {'0-4': 'd0', '5-9': 'd05', '10-14': 'd10', '15-19': 'd15', 
#         '20-24': 'd20', '25-29': 'd25', '30-34': 'd30', '35-39': 'd35', 
#         '40-44': 'd40', '45-49': 'd45', '50-54': 'd50', '55-59': 'd55', 
#         '60-64': 'd60', '65-69': 'd65', '70-74': 'd70', '75-79': 'd75', 
#         '80-84': 'd80', '85-89': 'd85', '90-94': 'd90', '95-99': 'd95', 
#         '100+': 'd95'}
ages = {'0-4': 'd0', '5-9': 'd5', '10-14': 'd10', '15-19': 'd15', 
        '20-24': 'd20', '25-29': 'd25', '30-34': 'd30', '35-39': 'd35', 
        '40-44': 'd40', '45-49': 'd45', '50-54': 'd50', '55-59': 'd55', 
        '60-64': 'd60', '65-69': 'd65', '70-74': 'd65', '75-79': 'd65', 
        '80-84': 'd65', '85-89': 'd65', '90-94': 'd65', '95-99': 'd65', 
        '100+': 'd65'}

# def changevalue(x):
#     return x * 1000

In [4]:
print('Please calculate the numbers and verify!')
len(df_input_pop), len(df_input_pop['LocID'].unique()), len(df_input_pop['Time'].unique()), len(df_input_pop['AgeGrp'].unique())

Please calculate the numbers and verify!


(1404753, 443, 151, 21)

In [5]:
df = df_input_pop[['LocID', 'Location', 'Variant', 'Time', 'AgeGrp', 'PopMale', 'PopFemale']]
# df = df[df['Variant']=='Medium']
df = df.rename(columns={"Time": "year", "AgeGrp": 'age'})
df['age'] = df['age'].apply(lambda x: ages[x])

df_male = df[['LocID', 'Location', 'Variant', 'year', 'age', 'PopMale']]
df_male['sex'] = 'M'
df_male = df_male.rename(columns={"PopMale": "population"})

df_female = df[['LocID', 'Location', 'Variant', 'year', 'age', 'PopFemale']]
df_female['sex'] = 'F'
df_female = df_female.rename(columns={"PopFemale": "population"})

df_combined = pd.concat([df_male, df_female])

gb = df_combined.groupby(['LocID', 'sex', 'age', 'year']).sum()
print(len(gb))
gb = gb.reset_index()

1873004


In [6]:
dff = pd.pivot_table(gb, columns=['year'], index=['LocID', 'sex', 'age'], values='population')
dff = dff.reset_index()

In [7]:
pop = dff.merge(UNcode_names, on='LocID').drop(columns=['LocID'])
pop = pop.dropna(subset=['Country Code'])
pop.replace('\s+', '', regex=True, inplace=True)
pop = pop[pop['Country Code'] != '']
pop = pop.set_index(['Country Code', 'sex', 'age'])
pop = (pop * 1000).astype('int')
pop.reset_index().to_csv(save_file_name, index=False)

In [8]:
import pandas as pd
def check_countries(filename):
    df = pd.read_csv(filename)
    countries = df['Country Code'].unique()
    country_names = pd.read_csv('../../../data/dl1_countrycodeorg_country_name.csv')
    GBD_countries = country_names[country_names["country"].notnull()]['Country Code']
    WB_countries = country_names[country_names["WBCountry"].notnull()]['Country Code']
    plus_GBD = sorted(list(set(countries) - set(GBD_countries)))
    sub_GBD = sorted(list(set(GBD_countries) - set(countries)))
    plus_WB = sorted(list(set(countries) - set(WB_countries)))
    sub_WB = sorted(list(set(WB_countries) - set(countries)))

    print ('Subtraction from WB:', sub_WB, len(sub_WB))
    print ('Plus from WB:', plus_WB, len(plus_WB))
    print ('Subtraction from GBD :', sub_GBD, len(sub_GBD))
    print ('Plus from GBD :', plus_GBD, len(plus_GBD))
check_countries(save_file_name)

Subtraction from WB: ['AND', 'ASM', 'BMU', 'CYM', 'DMA', 'FRO', 'GIB', 'GRL', 'IMN', 'KNA', 'LIE', 'MAF', 'MCO', 'MHL', 'MNP', 'NRU', 'PLW', 'SMR', 'SXM', 'TCA', 'TUV', 'VGB', 'XKX'] 23
Plus from WB: ['ESH', 'GLP', 'GUF', 'MTQ', 'MYT', 'PSE', 'REU', 'TWN'] 8
Subtraction from GBD : ['AND', 'ASM', 'BMU', 'COK', 'DMA', 'GRL', 'KNA', 'MCO', 'MHL', 'MNP', 'NIU', 'NRU', 'PLW', 'SMR', 'TKL', 'TUV'] 16
Plus from GBD : ['ABW', 'CHI', 'CUW', 'ESH', 'GLP', 'GUF', 'HKG', 'MAC', 'MTQ', 'MYT', 'NCL', 'PYF', 'REU'] 13


## Population Total

### We do not sum up all population by sex and age. All countries have total population.

In [9]:
print('Please calculate the numbers and verify!')
df_tmp = df_input_pop_total[df_input_pop_total['Variant']=='Medium']
print(len(df_tmp), len(df_tmp['Variant'].unique()), len(df_tmp['LocID'].unique()), len(df_tmp['Time'].unique()))


Please calculate the numbers and verify!
72027 1 477 151


In [10]:
pop_total = df_tmp.merge(UNcode_names, on='LocID')
pop_total = pop_total[['Country Code', 'Time', 'PopTotal']]
pop_total = pop_total.dropna(subset=['Country Code'])
pop_total.replace('\s+', '', regex=True, inplace=True)
pop_total = pop_total[pop_total['Country Code'] != '']
pop_total = pd.pivot_table(pop_total, columns=['Time'], index=['Country Code'], values='PopTotal')
pop_total = (pop_total * 1000).astype('int')
pop_total.reset_index().to_csv(save_file_name_total, index=False)

In [11]:
check_countries(save_file_name_total)

Subtraction from WB: ['XKX'] 1
Plus from WB: ['AIA', 'BES', 'BLM', 'COK', 'ESH', 'FLK', 'GLP', 'GUF', 'MSR', 'MTQ', 'MYT', 'NIU', 'PSE', 'REU', 'SHN', 'SPM', 'TKL', 'TWN', 'VAT', 'WLF'] 20
Subtraction from GBD : [] 0
Plus from GBD : ['ABW', 'AIA', 'BES', 'BLM', 'CHI', 'CUW', 'CYM', 'ESH', 'FLK', 'FRO', 'GIB', 'GLP', 'GUF', 'HKG', 'IMN', 'LIE', 'MAC', 'MAF', 'MSR', 'MTQ', 'MYT', 'NCL', 'PYF', 'REU', 'SHN', 'SPM', 'SXM', 'TCA', 'VAT', 'VGB', 'WLF'] 31
